In [10]:
import os

from mistralai import Mistral

api_keys = os.environ["MISTRAL_API_KEYS"]
model = "mistral-large-2411"

clients = [
    Mistral(
        api_key=api_key,
    )
    for api_key in api_keys.split(",")
]

for client in clients:
    # Check API is alive
    chat_response = client.chat.complete(
        model=model,
        messages=[
            {
                "role": "user",
                "content": "What is the best French cheese?",
            },
        ],
        max_tokens=10,
    )
    print(chat_response.model, chat_response.choices[0].message.content)

mistral-large-2411 Choosing the "best" French cheese can be
mistral-large-2411 Determining the "best" French cheese can


In [11]:
from time import sleep

from mistralai import SDKError
from openai import RateLimitError

SLEEP_DURATION = 1.2
if len(clients) == 2:
    SLEEP_DURATION = 0.5
if len(clients) >= 3:
    SLEEP_DURATION = 0.2

print("Sleep duration:", SLEEP_DURATION)


def wait(duration=SLEEP_DURATION):
    sleep(duration)


api_limit_hits_by_client_ids = {}


def init_api_limits() -> None:
    global api_limit_hits_by_client_ids

    api_limit_hits_by_client_ids = {}
    for i in range(len(clients)):
        api_limit_hits_by_client_ids[i] = 0


request_id = 0


def repeat_if_hit_api_limit(f):  # (1)
    def wrapper(*args, **kw):  # (2)
        global api_limit_hits_by_client_ids

        while True:
            try:
                return f(*args, **kw)
            except RateLimitError:
                client_id = request_id % len(clients)
                api_limit_hits_by_client_ids[client_id] += 1

                total_hits = 0
                for value in api_limit_hits_by_client_ids.values():
                    total_hits += value

                if (total_hits % 10) == 0:
                    print(f"API limit hit {total_hits} times. Details: {api_limit_hits_by_client_ids}")
                wait(2)
            except SDKError as e:
                if e.status_code == 429:
                    client_id = request_id % len(clients)
                    api_limit_hits_by_client_ids[client_id] += 1

                    total_hits = 0
                    for value in api_limit_hits_by_client_ids.values():
                        total_hits += value

                    if (total_hits % 10) == 0:
                        print(f"API limit hit {total_hits} times. Details: {api_limit_hits_by_client_ids}")
                    wait(2)
                else:
                    raise e
            except Exception as e:
                print("repeat_if_hit_api_limit -> unknown error", e)
                wait(60)

    return wrapper


@repeat_if_hit_api_limit
def query_model(messages):
    global request_id
    # print(request_id % len(clients))
    client = clients[request_id % len(clients)]
    request_id += 1
    response = client.chat.complete(model=model, messages=messages)
    return response

Sleep duration: 0.5


In [12]:
import os
import sys

dir2 = os.path.abspath("")
dir1 = os.path.dirname(dir2)
if dir1 not in sys.path:
    sys.path.append(dir1)

import importlib

import utils.prompt as prompt

# Required to purge the module cache and use the latest version after an update
importlib.reload(prompt)

<module 'utils.prompt' from '/Users/aigoncharov/dev/sktech/phi-4/utils/prompt.py'>

In [13]:
import csv
import os.path
import re

import pandas as pd
from tqdm import tqdm

import utils.prompt as prompt

# difficulty = ["middle_school", "high_school", "undergraduate", "postgraduate", "phd"]
ratings = list(range(1, 11, 1))

invalid_complexities = 0
invalid_ratings = 0


def model_as_judge(df, index, system_prompt, user_prompt, answer):
    global invalid_ratings

    chat_response = query_model(
        [
            {
                "role": "system",
                "content": 'Please act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user request displayed below. Your evaluation should consider factors such as the following all the settings in the system prompt, correspondences to the context of the user, the helpfulness, relevance and accuracy. Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, you must rate the response on a scale of 1 to 10 by strictly following this format: "[[rating]]", for example:"Rating: [[6]]".',
            },
            {
                "role": "user",
                "content": f"""
                [Instructions for Assistant]
                {system_prompt}
                [End of Instructions for Assistant]

                [Question]
                {user_prompt}
                [End of Question]

                [The Start of Assistant’s Answer]
                {answer}
                [The End of Assistant’s Answer]
                """,
            },
        ]
    )
    response = chat_response.choices[0].message.content
    # print(response)

    try:
        rating = re.search("\\[\\[(\\d+?)\\]\\]", response).group(1)
        # print(rating)
        rating_int = int(rating)
        if rating_int in ratings:
            df.at[index, "masj_num_rating"] = rating_int
        else:
            invalid_ratings += 1
    except:
        print(f"Could not extract rating from response:\n{response}\n")
        invalid_ratings += 1


def estimate_complextiy_with_model(df, index, system_prompt, user_prompt):
    global invalid_complexities

    chat_response = query_model(
        [
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": f"""
                [Question Start]
                {user_prompt}
                [Question End]
                """,
            },
        ]
    )
    response = chat_response.choices[0].message.content
    # print(response)

    try:
        complexity_str = re.search("\\[\\[(.+?)\\]\\]", response).group(1)
        # print(complexity_str)
        complexity = float(complexity_str)

        if complexity >= 0.0 and complexity <= 1.0:
            df.at[index, "masj_num_complexity"] = complexity
        else:
            invalid_complexities += 1
    except:
        print(f"Could not extract complexity from response:\n{response}\n")
        invalid_complexities += 1

    return response


DUMP_EVERY = 50


def estimate_dataset(in_filename, out_filename, get_question_from_row, get_options_from_row):
    if os.path.isfile(out_filename):
        df = pd.read_csv(
            out_filename,
            sep="\t",
            header=0,
            quoting=csv.QUOTE_NONE,
            quotechar="",
            escapechar="\\",
        )
    else:
        try:
            df = pd.read_csv(
                in_filename,
                sep="\t",
                header=0,
            )
        except:
            df = pd.read_csv(
                in_filename,
                sep="\t",
                header=0,
                quoting=csv.QUOTE_NONE,
                quotechar="",
                escapechar="\\",
            )

    global invalid_complexities
    global invalid_ratings
    invalid_complexities = 0
    invalid_ratings = 0
    init_api_limits()

    if "masj_num_complexity" not in df.columns:
        df["masj_num_complexity"] = -1.0
    if "masj_num_rating" not in df.columns:
        df["masj_num_rating"] = 0

    meaningful_iteration = 0
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        if (
            isinstance(df.at[index, "masj_num_complexity"], float)
            and df.at[index, "masj_num_complexity"] >= 0.0
            and df.at[index, "masj_num_complexity"] <= 1.0
            and df.at[index, "masj_num_rating"] in ratings
        ):
            continue

        meaningful_iteration += 1

        complexity_system_prompt = 'You are an expert in the topic of the question. Please act as an impartial judge and evaluate the complexity of the multiple-choice question with options below. Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, you must not answer the question. You must rate the question complexity as a number from 0 to 1 following the following scale as a reference: middle_school - 0.0-0.2, high_school - 0.2-0.4, undergraduate - 0.4-0.6, postgraduate - 0.6-0.8, phd - 0.8-1.0. You must return the complexity by strictly following this format: "[[complexity]]", for example: "Complexity: [[0.55]]", which corresponds to the undergraduate level.'
        complexity_user_prompt = prompt.get_user_prompt(get_question_from_row(row), get_options_from_row(row))
        # print(complexity_user_prompt)

        response_complexity = estimate_complextiy_with_model(
            df, index, complexity_system_prompt, complexity_user_prompt
        )
        wait()

        model_as_judge(df, index, complexity_system_prompt, complexity_user_prompt, response_complexity)
        wait()

        if meaningful_iteration % DUMP_EVERY == 0:
            df.to_csv(out_filename, sep="\t", quoting=csv.QUOTE_NONE, quotechar="", escapechar="\\", index=False)
            total_hits = 0
            for value in api_limit_hits_by_client_ids.values():
                total_hits += value
            print(f"Over {meaningful_iteration} iterations we hit {total_hits} API limits")

    df.to_csv(out_filename, sep="\t", quoting=csv.QUOTE_NONE, quotechar="", escapechar="\\", index=False)
    print(
        f"Processed dataset {out_filename}. Total entries: {df.shape[0]}. Invalid complexities: {invalid_complexities}. Invalid ratings: {invalid_ratings}"
    )
    return df

In [14]:
# # MMLU
# import ast

# estimate_dataset(
#     in_filename="../data/mmlu_pro_stem_w_maj_w_entropyphi4.tsv",
#     out_filename="../data/mmlu_pro_stem_w_numerical_maj_w_entropyphi4.tsv",
#     get_question_from_row=lambda row: row["question"],
#     get_options_from_row=lambda row: ast.literal_eval(row["options"]),
# )

In [15]:
# # ARC CH


# def get_options_arc(row):
#     try:
#         options_len = int(row["leng"])
#         options_str = row["text"]
#         options_str_without_newline = options_str.replace("\n", "")
#         options_str_without_brackets = options_str_without_newline[1:-1]
#         options_split = options_str_without_brackets.split("' '")
#         # Remove leading and trailing quotes from first and last options
#         options_split[0] = options_split[0][1:]
#         options_split[-1] = options_split[-1][:-1]
#         # print(options_split, options_len)
#         assert len(options_split) == options_len
#         for option in options_split:
#             assert len(option) > 0
#         return options_split
#     except AssertionError as e:
#         print(f"get_options_arc: {row['id']} -> AssertionError: {e}")
#         raise e


# estimate_dataset(
#     in_filename="../data/arc_ch_train_w_maj_complexity.tsv",
#     out_filename="../data/arc_ch_train_w_numerical_maj_complexity.tsv",
#     get_question_from_row=lambda row: row["question"],
#     get_options_from_row=get_options_arc,
# )
# estimate_dataset(
#     in_filename="../data/arc_ch_test_w_maj_complexity.tsv",
#     out_filename="../data/arc_ch_test_w_numerical_maj_complexity.tsv",
#     get_question_from_row=lambda row: row["question"],
#     get_options_from_row=get_options_arc,
# )
# estimate_dataset(
#     in_filename="../data/arc_ch_validation_w_maj_complexity.tsv",
#     out_filename="../data/arc_ch_validation_w_numerical_maj_complexity.tsv",
#     get_question_from_row=lambda row: row["question"],
#     get_options_from_row=get_options_arc,
# )

In [16]:
# # SCIQ
# from random import shuffle


# def get_options_sciq(row):
#     try:
#         options_len = 4
#         correct_option = row["correct"]
#         other_options = [row["incorrect1"], row["incorrect2"], row["incorrect3"]]
#         all_options = [correct_option] + other_options
#         shuffle(all_options)

#         assert len(all_options) == options_len
#         for option in all_options:
#             assert len(option) > 0
#         return all_options
#     except AssertionError as e:
#         print(f"get_options_sciq: {row['id']} -> AssertionError: {e}")
#         raise e


# estimate_dataset(
#     in_filename="../data/sciq_train.tsv",
#     out_filename="../data/sciq_train_w_maj_complexity.tsv",
#     get_question_from_row=lambda row: row["question"],
#     get_options_from_row=get_options_sciq,
# )
# estimate_dataset(
#     in_filename="../data/sciq_test.tsv",
#     out_filename="../data/sciq_test_w_maj_complexity.tsv",
#     get_question_from_row=lambda row: row["question"],
#     get_options_from_row=get_options_sciq,
# )
# estimate_dataset(
#     in_filename="../data/sciq_validation.tsv",
#     out_filename="../data/sciq_validation_w_maj_complexity.tsv",
#     get_question_from_row=lambda row: row["question"],
#     get_options_from_row=get_options_sciq,
# )


In [ ]:
# # GPQA
# from random import shuffle


# def get_options_gpqa(row):
#     try:
#         options_len = 4
#         correct_option = row["correct"]
#         other_options = [row["incorrect1"], row["incorrect2"], row["incorrect3"]]
#         all_options = [correct_option] + other_options
#         shuffle(all_options)

#         # print(all_options)

#         assert len(all_options) == options_len
#         for option in all_options:
#             assert len(option) > 0
#         return all_options
#     except AssertionError as e:
#         print(f"get_options_sciq: {row['id']} -> AssertionError: {e}")
#         raise e


# estimate_dataset(
#     in_filename="../data/gpqa_train.tsv",
#     out_filename="../data/gpqa_w_numerical_maj_complexity.tsv",
#     get_question_from_row=lambda row: row["question"],
#     get_options_from_row=get_options_gpqa,
# )

 65%|██████▍   | 291/448 [02:04<01:32,  1.70it/s]

Could not extract rating from response:
The assistant's response correctly identifies that the question involves finding the least common multiple (LCM) of the given orbital periods to determine the periodicity of a triple transit. This requires an understanding of mathematical concepts typically covered in high school or early undergraduate levels. The response is clear, concise, and relevant to the question's complexity.

Complexity: [[0.4]]



 71%|███████   | 319/448 [02:24<01:20,  1.61it/s]

Could not extract rating from response:
The assistant's response provides a clear and concise explanation of the knowledge domains and specific skills required to address the problem. The response highlights that the question involves understanding the concepts of wave functions, creation and annihilation operators, and energy calculations in the context of a quantum harmonic oscillator. This evaluation accurately reflects the complexity of the question, which aligns with advanced undergraduate or early postgraduate level quantum mechanics.

Complexity: [[0.65]]



100%|██████████| 448/448 [02:54<00:00,  2.57it/s]

Processed dataset ../data/gpqa_w_numerical_maj_complexity.tsv. Total entries: 448. Invalid complexities: 0. Invalid ratings: 2


,question,correct,incorrect1,incorrect2,incorrect3,explanation,topic,subject,difficulty,masj_num_complexity,masj_num_rating
0,"A large gene has dozens of exons, of which the...",R-loops,lariat,poly(A) tail,antisense,The text describes the dystrophin gene and the...,Molecular Biology,Biology,Hard undergraduate level (could be a question ...,0.83,10
1,Two quantum states with energies E1 and E2 hav...,10^-4 ev,10^-8 ev,10^-9 ev,10^-11 ev,"According to uncertainty principle, Delta E* D...",Physics (general),Physics,Hard undergraduate level (could be a question ...,0.50,10
2,trans-cinnamaldehyde was treated with methylma...,11,10,12,14,\ntrans-cinnamaldehyde was treated with methyl...,Organic Chemistry,Chemistry,Hard graduate level (could be a question on a ...,0.65,9
3,how many of the following compounds exhibit op...,4,3,5,6,the compounds\n1-methyl-4-(prop-1-en-2-yl)cycl...,Organic Chemistry,Chemistry,Hard undergraduate level (could be a question ...,0.65,9
4,A coating is applied to a substrate resulting ...,124°,129°,134°,139°,"In the Cassie-Baxter state, droplets are in co...",Chemistry (general),Chemistry,Hard undergraduate level (could be a question ...,0.70,9
...,...,...,...,...,...,...,...,...,...,...,...
443,Planet hunters decided to conduct follow-up ob...,~ 71.1 days,~ 56.2 days,~ 28.1 days,~ 18.7 days\n,"If the inclination is 90 degrees, then sin(i) ...",Astrophysics,Physics,Hard graduate level (could be a question on a ...,0.55,9
444,You are synthesizing a phosphodiester from two...,Two peaks changing to a single peak,A single peak changing to another single peak,A single peak changing to two peaks,Two peaks changing to three peaks,The final reaction converts a dialkyl chloroph...,Organic Chemistry,Chemistry,Post-graduate level or harder (only individual...,0.70,10
445,A coating is applied to a substrate resulting ...,"Water = 128°, Oil = 48°, low tilt angle","Water = 129°, Oil = 98°, low tilt angle","Water = 128°, Oil = 48°, high tilt angle","Water = 129°, Oil = 98°, high tilt angle",If the contact angle of a droplet on a smooth ...,Chemistry (general),Chemistry,Hard undergraduate level (could be a question ...,0.70,9
446,Two electromagnetic (em) fields are applied on...,11.54 e a0 E0,3 e a0 E0,5.1 e a0 E1,1.59 e a0 (E1 + E2),The dc electric field will dress the excited s...,Quantum Mechanics,Physics,Post-graduate level or harder (only individual...,0.85,9
